In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import copy
import torch.nn as nn

In [12]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval().cuda()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [ ]:
import json
file_path = 'transportation_sentences.json'

with open(file_path, 'r') as file:
    data = json.load(file)

print(data)

In [ ]:
import copy

features = {}

def forward_hook(module, input, output):
    features["decoder15"] = copy.deepcopy(output[0].cpu().detach() \
                                      if isinstance(output, (tuple, list)) else output.cpu().detach())
    features["decoder15"] = features["decoder15"].numpy().tolist()



hook_handle = model.model.layers[14].register_forward_hook(forward_hook)

In [ ]:
def generate_story(query = "I saw a Truck on the way to school."):

    prompt = f"Give me a short story that begins with: {query} \n\n"
    prompt += "Try to include more transportation-related words in the story.\n\n"
    prompt += "The story should contain at most 50 words"

    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    dummy_input = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    eos_token_id = tokenizer.eos_token_id

    token_feature = []
    for i in range(150):
        out = model(dummy_input)
        
        next_token_id = torch.argmax(out.logits[:, -1, :], dim=-1).unsqueeze(0)  # shape: (1, 1)
        
        dummy_input = torch.cat([dummy_input, next_token_id], dim=1)
        token_feature.append([next_token_id.cpu().numpy().tolist()[0][0], features["decoder15"][0][-1]])

        if next_token_id.item() == eos_token_id:
            break
    if True:
        print(tokenizer.decode(dummy_input[0], skip_special_tokens=True))

    return token_feature

In [ ]:
stories = {}
# data = data[:2]
torch.cuda.empty_cache() 
with torch.no_grad():
    for obj in data:
        for k, sentences in obj.items():
            for sentence in sentences:
                output = generate_story(sentence)
                try:
                    stories[k][sentence] = output
                except:
                    stories[k] = {sentence: output}


In [ ]:
import json

file_path = "dataset.json"

with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(stories, f, indent=4, ensure_ascii=False)


In [ ]:
file_path = "./data/dataset.json"

with open(file_path, 'r') as file:
    stories = json.load(file)

print(f"Category: {'Car'}")
print(f"First sentence: {'The Car is parked outside the building.'}")

story = stories['Car']['The Car is parked outside the building.']
print(f"Story: {tokenizer.decode([token[0] for token in story], skip_special_tokens=True)}")
print(f"Tokens: {[token[0] for token in story]}")